In [1]:
import time
import numpy as np 
import pandas as pd
import os,sys,argparse
import re

def ptm_cdr_wseq(seq,match_pattern):

    '''
    Cdrs
    AA such like "NG" "NS" "DG" on CDRs ;PTM means post translational modification.
    match_patten = r'N[GS]|DG'


    Wholeseq
    AA such like "N?S" "N?T" on the whole clone sequence.
    match_patten = r'N.[ST]'
    '''
    
    match = re.findall(match_pattern,seq)
    count = len(match)


    return count


def cal_ptms(line):
    
    cdr1counts = ptm_cdr_wseq(line[0],r'N[GS]|DG')
    cdr2counts = ptm_cdr_wseq(line[1],r'N[GS]|DG')
    cdr3counts = ptm_cdr_wseq(line[2],r'N[GS]|DG')
    wseqcounts = ptm_cdr_wseq(line[3],r'N.[ST]')
    counts = cdr1counts + cdr2counts + cdr3counts + wseqcounts
    
    return  cdr1counts,cdr2counts,cdr3counts,wseqcounts,counts
  


In [16]:
    
df_merge_clone = pd.DataFrame([['GFTFSRYS','ITSSSRYI','CARDPACGDYVYFDYW','771'],\
['NGNGCCG','ITTTTTNGSNGNGNGT','AATCITGSNGSECTW','589'],['NGNCCGT','ITTTTTNGSNGNGN\
GT','AATCITGWGNGSECTW','334'],['NGNCCNGT','INSTTTNGSNGNGNGT','AATCICGGSNGSECTW','234'\
]],columns = ['Cdr1','Cdr2','Cdr3','Counts'])
    
        

In [17]:
    print(df_merge_clone)

       Cdr1              Cdr2              Cdr3 Counts
0  GFTFSRYS          ITSSSRYI  CARDPACGDYVYFDYW    771
1   NGNGCCG  ITTTTTNGSNGNGNGT   AATCITGSNGSECTW    589
2   NGNCCGT  ITTTTTNGSNGNGNGT  AATCITGWGNGSECTW    334
3  NGNCCNGT  INSTTTNGSNGNGNGT  AATCICGGSNGSECTW    234


In [18]:
    df_merge_clone['seqlen']=df_merge_clone['Cdr3'].map(lambda x : len(x))
    df_merge_clone['Cnum']=df_merge_clone['Cdr3'].map(lambda x : x.count('C'))
    df_merge_clone

,Cdr1,Cdr2,Cdr3,Counts,seqlen,Cnum
0,GFTFSRYS,ITSSSRYI,CARDPACGDYVYFDYW,771,16,2
1,NGNGCCG,ITTTTTNGSNGNGNGT,AATCITGSNGSECTW,589,15,2
2,NGNCCGT,ITTTTTNGSNGNGNGT,AATCITGWGNGSECTW,334,16,2
3,NGNCCNGT,INSTTTNGSNGNGNGT,AATCICGGSNGSECTW,234,16,3


    #df_merge_clone['seqlen'].value_counts()

    #df_merge_clone.loc[df_merge_clone['seqlen'].idxmax()]

In [19]:
'''https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan'''
df_clone_filter = df_merge_clone[(df_merge_clone['seqlen']==df_merge_clone['seqlen'].max()) & (df_merge_clone['Cnum']==2)]
df_clone_filter

,Cdr1,Cdr2,Cdr3,Counts,seqlen,Cnum
0,GFTFSRYS,ITSSSRYI,CARDPACGDYVYFDYW,771,16,2
2,NGNCCGT,ITTTTTNGSNGNGNGT,AATCITGWGNGSECTW,334,16,2


In [20]:
df_clone_filter['Cdr3']

#df_clone_filter['Cdr3'].str.split('',expand=True).to_dict('index')

0    CARDPACGDYVYFDYW
2    AATCITGWGNGSECTW
Name: Cdr3, dtype: object

In [21]:
#转置行与列
df=df_clone_filter['Cdr3'].str.split('',expand=True).T
df

,0,2
0,,
1,C,A
2,A,A
3,R,T
4,D,C
5,P,I
6,A,T
7,C,G
8,G,W
9,D,G


In [9]:
#df_clone_filter
df_clone_filter['Cdr3'].map(lambda x : x.count('C'))

0    1
2    2
3    3
Name: Cdr3, dtype: int64

In [7]:

df_merge_clone['cdr1ptm'],df_merge_clone['cdr2ptm'],df_merge_clone['cdr3ptm'],df_merge_clone['wseqptm'],df_merge_clone['allptm']=zip(*df_merge_clone.apply(cal_ptms,axis=1))

df_merge_clone

,Cdr1,Cdr2,Cdr3,Counts,seqlen,cdr1ptm,cdr2ptm,cdr3ptm,wseqptm,allptm
0,GFTFSRYS,ITSSSRYI,CARDPAYGDYVYFDYW,771,16,0,0,0,0,0
1,NGNGCCG,ITTTTTNGSNGNGNGT,AATCITGCGSNGSECTW,589,17,2,4,1,0,7
2,NGNCCGT,ITTTTTNGSNGNGNGT,AATCITGCGSNGSECTW,334,17,1,4,1,0,6
3,NGNCCNGT,INSTTTNGSNGNGNGT,AATCICGCGSNGSECTW,234,17,2,5,1,0,8
